In [ ]:
# # # !pip install sqlalchemy
# # # !pip install psycopg2-binary
# # # !pip install Flask-SQLAlchemy
# # # !pip3 install sqlalchemy
# # # !pip3 install psycopg2-binary
# # # !pip3 install Flask-SQLAlchemy

# !conda install -y sqlalchemy
# !conda install -y psycopg2-binary

# !conda install -y sqlalchemy
# !conda install -y psycopg2-binary
# !conda install -y Flask-SQLAlchemy

# !pip install python-dotenv

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

In [ ]:
os.getenv('PATH')

In [ ]:
os.getenv('POSTGRES_USER')

In [ ]:
host = 'database-1.ctshzzzgs2pl.eu-central-1.rds.amazonaws.com' # 'localhost'
port = 5432
database = 'country_vaccinations' # 'northwind'
# user = 'postgres' # os.getenv('POSTGRES_USER')
# password = 'random-forest-fennel' # os.getenv('POSTGRES_PASSWORD') # set in .env file
user = os.getenv('POSTGRES_USER')
password = os.getenv('POSTGRES_PASSWORD') # set in .env file
# password = 'changeme'
uri = f'postgresql://{user}:{password}@{host}:{port}/{database}'
uri

In [ ]:
from sqlalchemy import create_engine

engine = create_engine(uri, echo=False)

In [ ]:
from sqlalchemy import Table
from sqlalchemy import inspect
from sqlalchemy import MetaData

def drop_table_if_exists(table_name):
    if inspect(engine).has_table(table_name):
        table = Table(table_name, MetaData(engine), autoload_with=engine)
        table.drop()

In [ ]:
import pandas as pd

# reload tables from csv files
data_dir= './var/data/country_vaccinations/'
files = os.listdir(data_dir)
for file in files:
    full_path = data_dir + file
    table_name = file.split('.')[0]
    ext = file.split('.')[1]
    if ext == 'csv':
        print('from %s file -> to "%s" table' % (full_path, table_name))
        drop_table_if_exists(table_name)
        df = pd.read_csv(full_path, index_col=0)
        df.to_sql(table_name, engine)

## 5. Analyze The Northwind Dataset

In [ ]:
# Get the names and the quantities in stock for each product.
query = 'SELECT p."productName", p."unitsInStock" FROM products AS p;'
df = pd.read_sql(query, engine)
df.head()

In [ ]:
# Get a list of current products (Product ID and name).
query = 'SELECT p."productID", p."productName" FROM products AS p;'
df = pd.read_sql(query, engine)
df.head()

In [ ]:
# Get a list of the most and least expensive products (name and unit price).
limit = 3
products_max = pd.read_sql(
    ('SELECT p."productName", p."unitPrice" FROM products AS p ORDER BY p."unitPrice" DESC LIMIT %d;' % limit), engine)
products_min = pd.read_sql(
    ('SELECT p."productName", p."unitPrice" FROM products AS p ORDER BY p."unitPrice" ASC LIMIT %d;' % limit), engine)
df = pd.concat([products_max, products_min], ignore_index=True)
df



In [ ]:
# Get products that cost less than $20.
products = pd.read_sql(
    ('SELECT p."productName", p."unitPrice" FROM products AS p WHERE p."unitPrice" > %d ORDER BY p."unitPrice" ASC;' % 20), engine)
products.head()

In [ ]:
# Get products that cost between $15 and $25.
products = pd.read_sql(
    ('SELECT p."productName", p."unitPrice" FROM products AS p WHERE p."unitPrice" BETWEEN %d AND %d ORDER BY p."unitPrice" ASC;' % (15, 25)), engine)
products.head()

In [ ]:
# Get products above average price.
avg_price = pd.read_sql(('SELECT AVG(p."unitPrice") AS avg_price FROM products AS p'), engine).loc[0, 'avg_price']
print('avg_price', avg_price)
products = pd.read_sql(
    ('SELECT p."productName", p."unitPrice" FROM products AS p WHERE p."unitPrice" > %d ORDER BY p."unitPrice" ASC;' % avg_price), engine)
products.head()

In [ ]:
# Find the ten most expensive products.
query = 'SELECT p.* FROM products AS p ORDER BY p."unitPrice" DESC LIMIT 10;'
df = pd.read_sql(query, engine)
df.head()

In [ ]:
# Get a list of discontinued products (Product ID and name).
query = 'SELECT p.* FROM products AS p WHERE p."discontinued" = 1;'
df = pd.read_sql(query, engine)
df.head()

In [ ]:
# Count current and discontinued products. ??? @TODO: ask what is "current product"
query = 'SELECT COUNT(p."discontinued"), p."discontinued" FROM products AS p GROUP BY p."discontinued";'
df = pd.read_sql(query, engine)
df.head()

In [ ]:
# Find products with less units in stock than the quantity on order.
query = 'SELECT p."productID", p."unitsInStock", p.* FROM products AS p WHERE p."unitsInStock" < p."unitsOnOrder";'
df = pd.read_sql(query, engine)
df.head()

In [ ]:
# Find the customer who had the highest order amount
query = """
SELECT 
    c."customerID", 
    c."companyName", 
    c."contactName", 
    c."contactTitle", 
    o."orderID", 
    SUM(od."unitPrice") AS "orderPrice"
FROM orders AS o
    INNER JOIN order_details AS od
        USING ("orderID")
    INNER JOIN customers AS c
        USING ("customerID" )
GROUP BY o."orderID", c."companyName", c."contactName", c."contactTitle", c."customerID"
ORDER BY "orderPrice" DESC
LIMIT 1
;
"""
df = pd.read_sql(query, engine)
df.head()

In [ ]:
# Get orders for a given employee and the according customer
query = 'SELECT c.* FROM customers AS c;'
df = pd.read_sql(query, engine)
df.head()

In [ ]:
# Find the hiring age of each employee
query = 'SELECT c.* FROM customers AS c;'
df = pd.read_sql(query, engine)
df.head()

In [ ]:
# Create views and/or named queries for some of these queries
query = 'SELECT c.* FROM customers AS c;'
df = pd.read_sql(query, engine)
df.head()

# Large countries - lesson exercises

In [ ]:
table_name = 'large_countries'

drop_table_if_exists(table_name)
df = pd.read_csv('./var/data/large_countries_2015.csv', index_col=0)
df.to_sql(table_name, engine)